In [1]:
from django.http import StreamingHttpResponse
from django.shortcuts import render, redirect
import cv2
import sqlite3
import logging

logger = logging.getLogger(__name__)

camera = None

class VideoCamera:
    def __init__(self):
        self.video = cv2.VideoCapture(0)
        if not self.video.isOpened():
            logger.error("Failed to open camera")
        self.facedetect = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        try:
            self.recognizer = cv2.face.LBPHFaceRecognizer_create()
            self.recognizer.read('trainer.yml')
        except cv2.error as e:
            logger.error(f"Error initializing or loading recognizer: {e}")

    def __del__(self):
        self.video.release()

    def get_frame(self):
        success, img = self.video.read()
        if not success:
            logger.error("Failed to capture frame")
            return None

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = self.facedetect.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            id, conf = self.recognizer.predict(gray[y:y + h, x:x + w])
            profile = self.get_profile(id)

            if profile:
                text = f"ID: {profile[0]} - Name: {profile[2]}"
                cv2.putText(img, text, (x, y + h + 20), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 2)

        ret, jpeg = cv2.imencode('.jpg', img)
        if not ret:
            logger.error("Failed to encode frame to JPEG")
            return None

        return jpeg.tobytes()

    @staticmethod
    def get_profile(id):
        """Retrieve user profile by ID from the database."""
        try:
            conn = sqlite3.connect("sqlite.db")
            cursor = conn.execute("SELECT * FROM STUDENTS WHERE id=?", (id,))
            profile = cursor.fetchone()
            return profile
        except sqlite3.Error as e:
            logger.error(f"Database error: {e}")
            return None
        finally:
            conn.close()

def gen(camera):
    while True:
        frame = camera.get_frame()
        if frame is None:
            continue
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n\r\n')

def video_feed(request):
    global camera
    if camera is None:
        camera = VideoCamera()
    return StreamingHttpResponse(gen(camera), content_type='multipart/x-mixed-replace; boundary=frame')

def stop_camera(request):
    global camera
    if camera is not None:
        del camera
        camera = None
    return redirect('detection_index')

def index(request):
    conn = sqlite3.connect("sqlite.db")
    cursor = conn.execute("SELECT * FROM STUDENTS")
    students = cursor.fetchall()
    conn.close()
    return render(request, 'detection/index.html', {'students': students})


In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("sqlite.db")

# Create a cursor object
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS STUDENTS (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    grade TEXT NOT NULL
)
''')

# Function to add a student
def add_student(id, name, age, grade):
    cursor.execute("SELECT id FROM STUDENTS WHERE id=?", (id,))
    if cursor.fetchone():
        print(f"Error: Student with ID {id} already exists.")
    else:
        cursor.execute('''
        INSERT INTO STUDENTS (id, name, age, grade) VALUES (?, ?, ?, ?)
        ''', (id, name, age, grade))
        conn.commit()
        print(f"Student with ID {id} added successfully.")

# Add students to the database
students = [
    (1, 'John Doe', 20, 'A'),
    (2, 'Jane Smith', 22, 'B'),
    (3, 'Alice Johnson', 21, 'A'),
    (4, 'Bob Brown', 23, 'C')
]

for student in students:
    add_student(*student)

# Function to retrieve and print student profile
def get_student_profile(id):
    cursor.execute("SELECT * FROM STUDENTS WHERE id=?", (id,))
    profile = cursor.fetchone()
    if profile:
        print("ID:", profile[0])
        print("Name:", profile[1])
        print("Age:", profile[2])
        print("Grade:", profile[3])
    else:
        print("Student not found.")

# Retrieve and print student profiles
get_student_profile(1)
get_student_profile(2)
get_student_profile(3)
get_student_profile(4)

# Close the connection
conn.close()


In [26]:
import sqlite3

def read_database(db_path):
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Execute a query to select all records from the STUDENTS table
        cursor.execute("SELECT * FROM STUDENTS")
        
        # Fetch all results from the executed query
        students = cursor.fetchall()
        
        # Fetch the column names from the cursor description
        column_names = [description[0] for description in cursor.description]
        
        # Print the column names
        print(column_names)
        
        # Print the results
        for student in students:
            print(student)
            
    except sqlite3.Error as e:
        print(f"An error occurred: {e.args[0]}")
        
    finally:
        # Close the database connection
        if conn:
            conn.close()

# Path to your SQLite database file
db_path = "sqlite.db"

# Call the function to read the database
read_database(db_path)


['id', 'FirstName', 'LastName', 'Gender', 'MedicalCondition', 'Address', 'EmergencyContact']
(1, 'Rob', 'Borom', 'M', 'HIV/AIDS', 'Siem Reap', 119)
(2, 'Nhor', 'Povketya', 'F', 'Diabetes', 'Kampot', 119)
(3, 'Sokhom', 'Kimheng', 'M', 'Autism', 'Siem Reap', 118)
(4, 'Ly', 'Hor', 'M', 'Autism', 'Phnom Penh', 118)
(5, 'Vay', 'Mithona', 'F', 'Diabetes', 'Phnom Penh', 119)
(6, 'Uk', 'Panapich', 'M', 'Hypertension', 'Siem Reap', 119)
(7, 'Phork', 'Norak', 'M', 'Hypertension', 'Kandal', 118)
(8, 'Prom', 'Sreyoun', 'F', 'Depression', 'Kampong Speu', 118)
(9, 'Pak', 'Chanshiraja', 'M', 'Depression', 'Phnom Penh', 119)
(10, 'Varn', 'Chorrathanak', 'F', 'Hypertension', 'Phnom Penh', 119)
(11, 'Sim', 'Lyheng', 'M', 'Cancers', 'Phnom Penh', 119)
(12, 'Checklim', 'Sunimithm', 'M', 'Diabetes', 'Phnom Penh', 118)
(13, 'Tan', 'Chesthareah', 'M', 'Autism', 'Battambang', 118)
(14, 'ssvhjsdiol', 'kjdnfjdk', 'kdfjnv', 'efoidvjldsip', 'lsdfneoisjldnvi', 32489)


In [6]:
import sqlite3

def read_database(db_path):
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Execute a query to select all records from the STUDENTS table
        cursor.execute("SELECT * FROM STUDENTS")
        
        # Fetch all results from the executed query
        students = cursor.fetchall()
        
        # Fetch the column names from the cursor description
        column_names = [description[0] for description in cursor.description]
        
        # Print the column names
        print(column_names)
        
        # Print the results
        for student in students:
            print(student)
            
    except sqlite3.Error as e:
        print(f"An error occurred: {e.args[0]}")
        
    finally:
        # Close the database connection
        if conn:
            conn.close()

def delete_row(db_path, student_id):
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Execute a query to delete the record with the specified ID
        cursor.execute("DELETE FROM STUDENTS WHERE id = ?", (student_id,))
        
        # Commit the changes
        conn.commit()
        
        print(f"Row with ID {student_id} has been deleted.")
        
    except sqlite3.Error as e:
        print(f"An error occurred: {e.args[0]}")
        
    finally:
        # Close the database connection
        if conn:
            conn.close()

# Path to your SQLite database file
db_path = "sqlite.db"

# Call the function to read the database
print("Database contents before deletion:")
read_database(db_path)

# Call the function to delete the row with ID 14
delete_row(db_path, 14)

# Call the function to read the database again to confirm deletion
print("\nDatabase contents after deletion:")
read_database(db_path)


Database contents before deletion:
['id', 'FirstName', 'LastName', 'Gender', 'MedicalCondition', 'Address', 'EmergencyContact']
(1, 'Rob', 'Borom', 'M', 'HIV/AIDS', 'Siem Reap', 119)
(2, 'Nhor', 'Povketya', 'F', 'Diabetes', 'Kampot', 119)
(3, 'Sokhom', 'Kimheng', 'M', 'Autism', 'Siem Reap', 118)
(4, 'Ly', 'Hor', 'M', 'Autism', 'Phnom Penh', 118)
(5, 'Vay', 'Mithona', 'F', 'Diabetes', 'Phnom Penh', 119)
(6, 'Uk', 'Panapich', 'M', 'Hypertension', 'Siem Reap', 119)
(7, 'Phork', 'Norak', 'M', 'Hypertension', 'Kandal', 118)
(8, 'Prom', 'Sreyoun', 'F', 'Depression', 'Kampong Speu', 118)
(9, 'Pak', 'Chansiharaja', 'M', 'Depression', 'Phnom Penh', 119)
(10, 'Varn', 'Chorrathanak', 'F', 'Hypertension', 'Phnom Penh', 119)
(11, 'Sim', 'Lyheng', 'M', 'Cancers', 'Phnom Penh', 119)
(12, 'Checklim', 'Sunimithm', 'M', 'Diabetes', 'Phnom Penh', 118)
(13, 'Tan', 'Chesthareah', 'M', 'Autism', 'Battambang', 118)
Row with ID 14 has been deleted.

Database contents after deletion:
['id', 'FirstName', 'LastN

In [10]:
import sqlite3

def initialize_database():
    conn = sqlite3.connect("sqlite.db")
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS STUDENTS (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        FirstName TEXT,
        LastName TEXT,
        Gender TEXT,
        MedicalCondition TEXT,
        Address TEXT,
        EmergencyContact INTEGER
    )
    """)
    
    # Data to be inserted
    students = [
        ('Rob', 'Borom', 'M', 'HIV/AIDS', 'Siem Reap', 119),
        ('Nhor', 'Povketya', 'F', 'Diabetes', 'Kampot', 119),
        ('Sokhom', 'Kimheng', 'M', 'Autism', 'Siem Reap', 118),
        ('Ly', 'Hor', 'M', 'Autism', 'Phnom Penh', 118),
        ('Vay', 'Mithona', 'F', 'Diabetes', 'Phnom Penh', 119),
        ('Uk', 'Panapich', 'M', 'Hypertension', 'Siem Reap', 119),
        ('Phork', 'Norak', 'M', 'Hypertension', 'Kandal', 118),
        ('Prom', 'Sreyoun', 'F', 'Depression', 'Kampong Speu', 118),
        ('Pak', 'Chanshiraja', 'M', 'Depression', 'Phnom Penh', 119),
        ('Varn', 'Chorrathanak', 'F', 'Hypertension', 'Phnom Penh', 119),
        ('Sim', 'Lyheng', 'M', 'Cancers', 'Phnom Penh', 119),
        ('Checklim', 'Sunimithm', 'M', 'Diabetes', 'Phnom Penh', 118),
        ('Tan', 'Chesthareah', 'M', 'Autism', 'Battambang', 118)
    ]
    
    # Insert data
    cursor.executemany("""
    INSERT INTO STUDENTS (FirstName, LastName, Gender, MedicalCondition, Address, EmergencyContact) 
    VALUES (?, ?, ?, ?, ?, ?)
    """, students)
    
    conn.commit()
    conn.close()

# Initialize and populate the database
initialize_database()

print("Database initialized and populated with data.")


Database initialized and populated with data.


In [14]:
import sqlite3

def initialize_database():
    conn = sqlite3.connect("sqlite.db")
    conn.execute("""
    CREATE TABLE IF NOT EXISTS STUDENTS (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        FirstName TEXT,
        LastName TEXT,
        Gender TEXT,
        MedicalCondition TEXT,
        Address TEXT,
        EmergencyContact INTEGER
    )
    """)
    conn.close()

if __name__ == "__main__":
    initialize_database()
    print("Database initialized successfully.")


Database initialized successfully.


In [24]:
import sqlite3
import django
from django.conf import settings
from detection.models import Student

# Set up Django environment
if not settings.configured:
    settings.configure(
        DEBUG=True,
        INSTALLED_APPS=[
            'detection',
        ],
        DATABASES={
            'default': {
                'ENGINE': 'django.db.backends.sqlite3',
                'NAME': 'db.sqlite3',
            }
        }
    )
django.setup()

def sync_data():
    try:
        conn = sqlite3.connect("sqlite.db")
        cursor = conn.cursor()
        cursor.execute("SELECT id, FirstName, LastName, Gender, MedicalCondition, Address, EmergencyContact FROM STUDENTS")
        sqlite_students = cursor.fetchall()
        conn.close()

        for student in sqlite_students:
            _, created = Student.objects.get_or_create(
                id=student[0],
                defaults={
                    'FirstName': student[1],
                    'LastName': student[2],
                    'Gender': student[3],
                    'MedicalCondition': student[4],
                    'Address': student[5],
                    'EmergencyContact': student[6],
                }
            )
        print("Data synchronization successful.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    sync_data()


RuntimeError: You cannot use AsyncToSync in the same thread as an async event loop - just await the async function directly.